# AP3 - Pattern Recognition
Implementation of a decision tree for classifying a database with categorical attributes.

> Name: Jonas Carvalho Fortes

> Mat: 494513

## Load Dataset

In [37]:
import pandas as pd
from scipy.io import loadmat

dataset = loadmat('data/Dataset.mat')
dataset = pd.DataFrame(dataset['Dataset'])
print(f'Dataset shape: {dataset.shape}')



Dataset shape: (876, 4)


In [38]:
print('Dataset:')
dataset

Dataset:


,0,1,2,3
0,1,0,0,1
1,0,0,0,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,1
...,...,...,...,...
871,1,1,0,1
872,1,1,0,1
873,1,1,0,1
874,1,0,1,1


## Helper functions and classes

In [55]:
def entropy(labels):
    """
    Calcula a entropia de um conjunto de rótulos.
    
    Args:
        labels (np.ndarray): Array contendo os rótulos dos dados (0 ou 1).
    
    Returns:
        float: Entropia do conjunto de rótulos.
    """
    # Calcula a frequência de cada classe
    values, counts = np.unique(labels, return_counts=True)
    
    # Calcula a probabilidade de cada classe
    probabilities = counts / len(labels)
    
    # Calcula a entropia usando a fórmula da entropia
    entropy_value = -np.sum(probabilities * np.log2(probabilities + EPSILON))
    
    return entropy_value

def information_gain(original_labels, left_labels, right_labels):
    """
    Calcula o ganho de informação de uma divisão dos dados.
    
    Args:
        original_labels (np.ndarray): Rótulos antes da divisão.
        left_labels (np.ndarray): Rótulos do subconjunto à esquerda.
        right_labels (np.ndarray): Rótulos do subconjunto à direita.
    
    Returns:
        float: Ganho de informação da divisão.
    """
    # Entropia antes da divisão
    original_entropy = entropy(original_labels)
    
    # Calcula a entropia ponderada após a divisão
    total_len = len(original_labels)
    left_prob = len(left_labels) / total_len
    right_prob = len(right_labels) / total_len
    
    # Entropia ponderada após a divisão
    weighted_entropy = (left_prob * entropy(left_labels)) + (right_prob * entropy(right_labels))
    
    # Ganho de informação
    gain = original_entropy - weighted_entropy
    
    return gain

def get_majority_class(labels):
    """
    Obtém a classe majoritária de um conjunto de rótulos.
    
    Args:
        labels (np.ndarray): Array contendo os rótulos dos dados (0 ou 1).
    
    Returns:
        int: A classe majoritária.
    """
    # Conta a frequência de cada classe
    values, counts = np.unique(labels, return_counts=True)
    
    # Retorna a classe com a maior frequência
    majority_class = values[np.argmax(counts)]
    
    return majority_class



In [57]:
import numpy as np 

# Menor valor de entropia possível que não seja zero (para evitar divisão por zero)
EPSILON = np.finfo('float32').eps

class Node:
    
    def __init__(self, left, right, data=None, feature_idx=None, 
                 feature_name=None, criterion_value=None,
                 _result=None, class_name=None):
        
        self.left: Node = left
        self.right: Node = right

        self.data = data
        self.feature_idx: int = feature_idx
        self.feature_name: str = feature_name
        self.criterion_value: float = criterion_value
        self.n_sample: int = len(data) if data is not None else 0
        self._result = _result
        self.class_name: str = class_name

    def predict(self, x):
        if x[self.feature_idx] == 0:
            return self.left.predict(x)
        else:  # Aqui x[self.feature_idx] deve ser 1
            return self.right.predict(x)


class LeafNode(Node):
    def __init__(self, data, criterion_value, _result, class_name):
        super().__init__(None, None, data=data, 
                         criterion_value=criterion_value, 
                         _result=_result, class_name=class_name)

    def predict(self, X=None):
        return self._result
    
    
class DecisionTree: 
    
    def __init__(self, feature_names=[], class_names=[]):
        self.root: Node = Node(None, None)
        self.feature_names = feature_names
        self.class_names = class_names
    
    def fit(self, X: np.ndarray, y: np.ndarray):
         # Cria um array de dados que inclui atributos e rótulos
        data = np.hstack((X, y.reshape(-1, 1)))
        # Chama o método grow para construir a árvore
        self.root = self._grow(data)
    
    def predict(self, X):
        return self.root.predict(X)

    def _split_data(self, feature_data, labels):
        """
        Divide os dados com base nos valores categóricos (0 ou 1) do atributo.
        """
        # Dividindo à esquerda (valor 0)
        left_indices = feature_data == 0
        left_labels = labels[left_indices]

        # Dividindo à direita (valor 1)
        right_indices = feature_data == 1
        right_labels = labels[right_indices]

        return (left_labels, right_labels)
    
    def _best_split(self, feature_data, labels):
        """
        Avalia a entropia resultante da divisão dos dados para um determinado atributo categórico.
        """
        # Dividindo os dados com base no valor binário do atributo (0 ou 1)
        left_labels, right_labels = self._split_data(feature_data, labels)

        # Calcula a entropia da divisão
        cost_value = entropy(left_labels, right_labels)
        
        return cost_value
    
    def _best_feature(self, data, feature_idxs):
        """
        Identifica o melhor atributo binário para dividir os dados.
        """
        max_gain = -np.inf
        selected_feature = None

        for feature_idx in feature_idxs:
            # Obtém os dados do atributo
            feature_data = data[:, feature_idx]
            labels = data[:, -1]
            
            # Avalia o ganho de informação para o atributo atual
            left_labels, right_labels = self._split_data(feature_data, labels)
            gain = information_gain(labels, left_labels, right_labels)

            if gain > max_gain:
                max_gain = gain
                selected_feature = feature_idx
        
        return selected_feature, max_gain
    
    def _grow(self, data, used_features=None):
        """
        Constrói recursivamente a árvore de decisão.
        """
        if used_features is None:
            used_features = set()

        compute_criterion_value = entropy
        get_result = get_majority_class

        y = data[:, -1]
        criterion_value = compute_criterion_value(y)
        result = get_result(y)

        class_name = self.class_names[result] if self.class_names else f"{result:.4f}"

        # Critério de parada: parar se a entropia for zero (dados puros) ou se todos os atributos já tiverem sido usados
        if criterion_value < EPSILON or len(used_features) >= 3:
            return LeafNode(data, criterion_value=criterion_value, 
                            _result=result, class_name=class_name)

        # Seleciona o melhor atributo para divisão
        feature_idxs = [i for i in np.arange(data.shape[-1] - 1) if i not in used_features]
        selected_feature, _ = self._best_feature(data, feature_idxs)

        if selected_feature is None:
            return LeafNode(data, criterion_value=criterion_value, 
                            _result=result, class_name=class_name)

        used_features.add(selected_feature)

        # Divide os dados com base no atributo selecionado
        left_data = data[data[:, selected_feature] == 0]
        right_data = data[data[:, selected_feature] == 1]

        # Cria os nós filhos recursivamente
        left_node = self._grow(left_data, used_features=used_features)
        right_node = self._grow(right_data, used_features=used_features)

        return Node(left_node, 
                    right_node,
                    data, 
                    selected_feature, 
                    feature_name=self.feature_names[selected_feature] if any(self.feature_names) else "NA",
                    criterion_value=criterion_value,
                    _result=result, class_name=class_name)

In [58]:
# Separando atributos e rótulos
X = dataset.iloc[:, :-1].values  # Atributos (todas as colunas exceto a última)
y = dataset.iloc[:, -1].values   # Rótulo (última coluna)

# Nomes dos atributos e classes
feature_names = ["Empregado", "Devedor", "Salário acima de 5SM"]
class_names = ["não", "sim"]

# Criando e treinando a árvore de decisão
tree = DecisionTree(feature_names=feature_names, class_names=class_names)
tree.fit(X, y)

In [62]:
# Exemplo de previsão
test_data = np.array([
    [1, 0, 1],  # empregado: sim, devedor: não, salário acima de 5SM: sim
    [0, 1, 0],  # empregado: não, devedor: sim, salário acima de 5SM: não (estranho ser classe 'sim')
    [0, 0, 0],  # empregado: não, devedor: não, salário acima de 5SM: não
    [1, 1, 1],  # empregado: sim, devedor: sim, salário acima de 5SM: sim
    [1, 1, 0],  # empregado: sim, devedor: sim, salário acima de 5SM: não
])

# Prevendo com a árvore treinada
predictions = [tree.predict(x) for x in test_data]
print("Previsões:", [class_names[pred] for pred in predictions])

Previsões: ['sim', 'sim', 'não', 'sim', 'sim']
